In [1]:
import os
import zipfile
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mido import MidiFile, MidiTrack, Message
import matplotlib.pyplot as plt
from music21 import converter, stream
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pickle
import json
from matplotlib import image as mpimg
from tqdm import tqdm

In [ ]:
def check_gpu():
    physical_devices = tf.config.list_physical_devices('GPU')
    if physical_devices:
        print("GPU is available.")
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    else:
        print("No GPU available. Using CPU.")


def load_midi_files(file_directory):
    midi_files = []
    for root, dirs, files in os.walk(file_directory): 
        print(f"Checking folder: {root}") 
        for file in files:
            if file.endswith(".midi") or file.endswith(".mid"): 
                midi_files.append(os.path.join(root, file)) 
    return midi_files

def load_midi(file_path):
    midi = MidiFile(file_path)
    notes = []
    for i, track in enumerate(midi.tracks):
        for msg in track:
            if msg.type == 'note_on': 
                notes.append(msg.note)
    return notes


def pca_feature_extraction(data, n_components=32):
    print(f"Starting PCA with {n_components} components...")

    n_features = data.shape[1]  # Number of features (sequence length)
    n_components = min(n_components, n_features)
    
    pca = PCA(n_components=n_components)
    pca_result = pca.fit_transform(data)
    print("PCA has been completed.")
    return pca_result

def prepare_sequences(data, sequence_length=32):
    sequences = []
    labels = []
    for i in range(len(data) - sequence_length):
        seq = data[i:i+sequence_length]
        sequences.append(seq)
        labels.append(data[i+sequence_length])
    return np.array(sequences), np.array(labels)


def train_lstm(sequences, labels, input_shape, num_classes, epochs=20, batch_size=32):
    print(f"Starting LSTM model training with {epochs} epochs and batch size {batch_size}...")
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(128, input_shape=input_shape, return_sequences=True),
        tf.keras.layers.LSTM(64, return_sequences=False),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(sequences, labels, epochs=epochs, batch_size=batch_size, validation_split=0.2, callbacks=[early_stopping])
    print("LSTM training has been completed.")
    return model, history

def kmeans_clustering(data, n_clusters=128):
    print("Starting K-means clustering...")
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(data)
    print("K-means clustering has been completed.")
    return kmeans.predict(data)

def convert_to_midi(clusters, output_filename="output_midi.mid"):
    print(f"Converting clusters to MIDI: {output_filename}...")
    midi_file = MidiFile()
    track = MidiTrack()
    midi_file.tracks.append(track)
    for note in clusters:
        msg = Message('note_on', note=note, velocity=64, time=0)
        track.append(msg)
        msg = Message('note_off', note=note, velocity=64, time=100)
        track.append(msg)
    midi_file.save(output_filename)
    print(f"MIDI file saved as: {output_filename}")


def convert_midi_to_music_representation(midi_data):
    music_rep = converter.parse(midi_data)
    return music_rep


def export_sheet_music(sheet_music, output_format, filename):
    sheet_music.write(output_format, fp=filename)


def prepare_midi_data(midi_files):
    print("Preparing MIDI data...")
    sequences = []
    labels = []
    for midi_file in tqdm(midi_files, desc="Loading MIDI files"):
        midi_notes = load_midi(midi_file)  
        sequences_midi, labels_midi = prepare_sequences(midi_notes) 
        sequences.extend(sequences_midi)
        labels.extend(labels_midi)

    print("Applying PCA to the data...")
    sequences_pca = pca_feature_extraction(sequences)
    
    sequences_pca = np.expand_dims(np.array(sequences_pca), axis=-1)
    print("MIDI data preparation is complete.")
    return sequences_pca, np.array(labels)


def cache_midi_files(file_directory, cache_file):
    midi_files = load_midi_files(file_directory)
    with open(cache_file, 'wb') as f:
        pickle.dump(midi_files, f) 
    print(f"Saved {len(midi_files)} MIDI file paths to cache at {cache_file}.")
    return midi_files


def load_cached_midi_files(cache_file):
    if os.path.exists(cache_file):
        print(f"Loading cached MIDI files from: {cache_file}")
        with open(cache_file, 'rb') as f:
            midi_files = pickle.load(f) 
        print(f"Loaded {len(midi_files)} MIDI file paths from cache.")
        return midi_files
    else:
        print("No cache found. Loading MIDI files from the dataset...")
        return None


def save_model(model, model_path):
    model.save(model_path)
    print(f"Model saved at: {model_path}")

def save_results(val_loss, val_accuracy, results_path):
    results = {
        "validation_loss": val_loss,
        "validation_accuracy": val_accuracy
    }
    
    with open(results_path, 'w') as f:
        json.dump(results, f)
    print(f"Results saved at: {results_path}")

def load_model_from_disk(model_path):
    model = tf.keras.models.load_model(model_path)
    print(f"Model loaded from: {model_path}")
    return model

def plot_training_history(history):
    print("Plotting training history...")

    plt.figure(figsize=(14, 6))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

def generate_sheet_music_from_new_midi(input_midi_path, model, sequence_length=32):
    print("Generating sheet music from new MIDI file...")

    midi_notes = load_midi(input_midi_path)  
    input_sequences, _ = prepare_sequences(midi_notes, sequence_length)
    
    predicted_notes = model.predict(input_sequences)
    
    output_midi_path = 'predicted_output_midi.mid'
    generate_midi(predicted_notes[0], output_filename=output_midi_path)
    print(f"Generated MIDI file saved as: {output_midi_path}")

    music_representation = convert_midi_to_music_representation(output_midi_path)
    
    output_pdf_path = 'predicted_sheet_music.pdf'
    music_representation.write('pdf', fp=output_pdf_path)
    print(f"Sheet music has been exported to: {output_pdf_path}")

def main(extracted_folder, cache_file, model_save_path, results_save_path):

    check_gpu()

    midi_files = load_cached_midi_files(cache_file)
    
    if midi_files is None: 
        midi_files = []
        for year_folder in range(2004, 2019):  
            midi_folder = os.path.join(extracted_folder, f'{year_folder}')
            midi_files_in_year = load_midi_files(midi_folder) 
            midi_files.extend(midi_files_in_year)
        
        if len(midi_files) == 0:
            print("No MIDI files found. Please check the extraction and folder paths.")
            return

        print(f"Loaded {len(midi_files)} MIDI files from the MAESTRO dataset.")

        cache_midi_files(extracted_folder, cache_file)
    
    sequences, labels = prepare_midi_data(midi_files)
    
    train_sequences, val_sequences, train_labels, val_labels = train_test_split(sequences, labels, test_size=0.2, random_state=42)
    print(f"Training Sequences Shape: {train_sequences.shape}")
    print(f"Validation Sequences Shape: {val_sequences.shape}")

    input_shape = train_sequences.shape[1:] 
    num_classes = np.max(train_labels) + 1 

    model = train_lstm(train_sequences, train_labels, input_shape, num_classes, epochs=5, batch_size=16)

    val_loss, val_accuracy = model.evaluate(val_sequences, val_labels)
    print(f"Validation Loss: {val_loss}")
    print(f"Validation Accuracy: {val_accuracy}")
    
    save_model(model, model_save_path)
    save_results(val_loss, val_accuracy, results_save_path)

    clusters = kmeans_clustering(val_sequences)

    convert_to_midi(clusters)

    music_representation = convert_midi_to_music_representation("output_midi.mid")
    sheet_music = music_representation
    output_file_path_XML = "./resultXML/" + os.path.splitext(os.path.basename(extracted_folder))[0] + ".xml"
    output_file_path_PDF = "./resultPDF/" + os.path.splitext(os.path.basename(extracted_folder))[0] + ".pdf"
    export_sheet_music(sheet_music, "musicxml", output_file_path_XML)

    print(f"MIDI file has been converted and saved as 'output_midi.mid'.")
    print(f"Sheet music exported as MusicXML: {output_file_path_XML}")
    print(f"Sheet music exported as PDF: {output_file_path_PDF}")

    plot_training_history(model.history)

In [ ]:
extracted_folder = r'C:\Users\BoilingMachine\OneDrive\Desktop\Diploma\maestro-v3.0.0'

cache_file = r'C:\Users\BoilingMachine\OneDrive\Desktop\Diploma\maestro-v3.0.0\midi_files_cache.pkl'

model_save_path = r'./saved_model/my_model'
results_save_path = r'./saved_model/results.json'

main(extracted_folder, cache_file, model_save_path, results_save_path)